# 5.6 - Undervalued, Top-Rated Listings

This notebook serves to provide insight on undervalued and top-rated listings in the test set.

- *Undervalued* listings are defined as the listings for which the price is lower than the predicted value of the model.
- *Top-rated* listings are defined as the listings which have the highest positive sentimental score.

In the following section, the the most undervalued listings are identified in the test set:

## Identifying Undervalued Listings

As a first step, the employed python libraries are imported:

In [1]:
import pandas as pd
import numpy as np
import joblib as jl
import preprocessing as ppc

from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler

Next, the listings and the reviews are read. Moreover, the function `test_listing_ids` is defined which is later used to retrieve the listing IDs for all listings in the test set.

In [2]:
listings = pd.read_csv('../../data/listings.csv')
reviews1 = pd.read_csv('../../data/reviews1.csv')
reviews2 = pd.read_csv('../../data/reviews2.csv')
reviews3 = pd.read_csv('../../data/reviews3.csv')
reviews4 = pd.read_csv('../../data/reviews4.csv')

reviews = pd.concat([reviews1, reviews2, reviews3, reviews4])

In [4]:
def test_listing_ids(listings):
    """
    Retrieve test set listing IDs. This method applies to the input some of the 
    preprocessing steps which are necessary to correctly identify and retrieve the IDs.

    :param listings: Input DataFrame
    :return: pandas.Series containing listing IDs of test set.
    
    """
    
    data_copy = listings.copy()
    data_copy.drop(["neighbourhood_group_cleansed", "bathrooms", "calendar_updated"],
                   axis=1, inplace=True)
    data_copy.drop(["listing_url", "picture_url", "host_url", "host_thumbnail_url", "host_picture_url"],
                   axis=1, inplace=True)
    data_copy.drop(["scrape_id", "host_id"],
                   axis=1, inplace=True)
    data_copy.drop(["name", "description", "neighborhood_overview", "host_name", "host_about", "host_location",
                    "host_neighbourhood", "host_verifications", "neighbourhood", "license"],
                   axis=1, inplace=True)
    data_copy.drop(["last_scraped", "calendar_last_scraped"],
                   axis=1, inplace=True)
    
    data_copy = data_copy[~data_copy.iloc[:, 1:].duplicated()]
    
    data_copy = data_copy.dropna(thresh=data_copy.shape[1]-3)
    X_train, X_test = train_test_split(data_copy, test_size=0.3, random_state=42)
    
    return X_test["id"]

Having provided the necessary tools for the task in question, we proceed to load the trained model and the fitted scaler. We run the preprocessing pipeline in order to obtain `X_test` and `y_test`. 

A DataFrame `test_listings` is constructed which contains the retrieved listing IDs for the listings in the test set (`id`), the price value predicted by the model (`prediction`) and the listed price of the accommodation (`price`).

In [5]:
# Load the trained model and the fitted scaler.
model = jl.load('../fastapi/data/regressor.joblib')
scaler = jl.load('../fastapi/data/scaler.joblib')

# Obtain the test set.
X_train, X_test, y_train, y_test = ppc.run_preprocessing_pipeline(listings)

# Produce a DataFrame that contains only listing IDs along the the registered and predicted prices.
test_listings = pd.DataFrame(data=test_listing_ids(listings),columns=['id']);
test_listings['prediction'] = model.predict(X_test)
test_listings['price'] = y_test

Furthermore, the prediction error is computed and stored as a column in `test_listings`.

In [6]:
test_listings['prediction_error'] = test_listings['prediction'] - test_listings['price']

The 10 most undervalued listings are presented below:

In [7]:
test_listings.sort_values(["prediction_error"], ascending=False, ignore_index=True).head(10)

,id,prediction,price,prediction_error
0,28883633,961.227749,473.0,488.227749
1,33902024,390.944552,71.0,319.944552
2,478626,369.053455,120.0,249.053455
3,29951541,351.303464,110.0,241.303464
4,18064024,310.603013,83.0,227.603013
5,4812115,238.635238,48.0,190.635238
6,17186613,257.238749,70.0,187.238749
7,14864433,206.750919,20.0,186.750919
8,20271395,308.439627,175.0,133.439627
9,968409,204.171771,73.0,131.171771


## Identifying Top-Rated Listings
In this section, top-rated listings are identified based on sentimental analysis of the two most recent reviews for each listing.

For this task, the following libraries are imported:

In [8]:
import nltk
import string

from googletrans import Translator
from bs4 import BeautifulSoup
from datetime import datetime

Next, we retrieve the 1st and 2nd most recent reviews for each listing. Since it is not possible to directly group `reviews` by `listing_id`, then sort by `date` and obtain the two most recent reviews for each listing using pandas, a different process is followed.

This process involves creating two copies of `reviews`, from which only relevant reviews are kept. *Relevant* reviews are the reviews which refer to listings in the test set and which contain adequate information (e.g. more than 10 characters).

`reviews_most_recent1` is grouped by `listing_id` and the the first most recent review for all listings is obtained by using `max` as an aggregate function. Then, reviews present in `reviews_most_recent1` are excluded from `reviews_most_recent2`, which can then be used to obtain the second most recent review by following the same steps.

DataFrames `reviews_most_recent1` and `reviews_most_recent2` are then combined appropriately into `reviews_most_recent` to obtain the first and second most recent review comments in a single row for each listing.

In [9]:
# Define two copies of reviews
reviews_most_recent1 = reviews.copy()
reviews_most_recent2 = reviews.copy()

# Define a mask of relevant reviews, i.e. reviews that refer to the listings in the test set and which contain more than 10 chars.
relevant_reviews = reviews["listing_id"].isin(test_listings["id"]) & (reviews["comments"].str.len() > 10)

# The 'date' column is processed into a comparable data type so that the 'max' function can be used on it.
reviews_most_recent1["date"] = reviews_most_recent1["date"].apply( lambda x : datetime.strptime(x, '%Y-%m-%d') ) 
reviews_most_recent2["date"] = reviews_most_recent2["date"].apply( lambda x : datetime.strptime(x, '%Y-%m-%d') ) 

# Obtain most recent review for each listing.
reviews_most_recent1 = reviews_most_recent1[ relevant_reviews ].groupby("listing_id", as_index=False).max('date')

# Exclude the first most recent reviews for each listing from 'reviews_most_recent2'. Now, the second most recent review can be obtained.
reviews_most_recent2 = reviews_most_recent2[ relevant_reviews ]
reviews_most_recent2 = reviews_most_recent2[ ~reviews_most_recent2["id"].isin(reviews_most_recent1["id"]) ]
reviews_most_recent2 = reviews_most_recent2.groupby("listing_id", as_index=False).max('date')

# Using group_by causes non-numeric columns to be dropped from the dataframes. Retrieve the corresponding comments using inner joins.
reviews_most_recent1 = reviews_most_recent1.merge(right=reviews, how='inner', left_on='id', right_on='id')[['listing_id_x','id','comments']]
reviews_most_recent2 = reviews_most_recent2.merge(right=reviews, how='inner', left_on='id', right_on='id')[['listing_id_x','id','comments']]

# "Combine" the two most recent reviews and obtain a DataFrame which contains both reviews in a single row for each listing.
# This is done by applying a left join on 'reviews_most_recent1'.
reviews_most_recent = reviews.copy()
reviews_most_recent = reviews_most_recent1.merge(right=reviews_most_recent2, how='left', left_on='listing_id_x', right_on='listing_id_x')
reviews_most_recent = reviews_most_recent[[ 'listing_id_x', 'comments_x', 'comments_y' ]]

In this step, we check for any second most recent review being missing and we fill them using a dummy string.

Then, the most recent reviews are cleaned from any redundand HTML tags and punctuation marks that may cause problem when analysing the reviews.

In [10]:
# Fill missing second most recent review comments using dummy text.
reviews_most_recent["comments_y"] = reviews_most_recent["comments_y"].fillna("0")

# Process the first and second most recent reviews remove HTML tags and punctuation marks.
reviews_most_recent["comments_x"] = reviews_most_recent["comments_x"].apply( lambda x : BeautifulSoup(str(x)).get_text().translate(str.maketrans("","", string.punctuation) ) )
reviews_most_recent["comments_y"] = reviews_most_recent["comments_y"].apply( lambda x : BeautifulSoup(str(x)).get_text().translate(str.maketrans("","", string.punctuation) ) )

In the following snippets, all selected reviews are translated into English (if they are written in a different language) using a Google Translator API:

In [ ]:
transl = Translator()
transl.raise_Exception = True

translated_comments_x = []

for comment in reviews_most_recent["comments_x"]:
    # Translate the review comment and obtain its text
    trans = transl.translate(str(comment), dest='en').text
    # Store into 'translated_comments_x'
    translated_comments_x.append(trans)

In [ ]:
transl = Translator()
transl.raise_Exception = True

translated_comments_y = []
for comment in reviews_most_recent["comments_y"]:
    # Translate the review comment and obtain its text
    trans = transl.translate(str(comment), dest='en').text
    # Store into 'translated_comments_x'
    translated_comments_y.append(trans)

Next, the two reviews for each listing are concatenated and stored in `reviews_most_recent`. This allows us to utilize Microsoft Azure's Cognitive Services using less elements for analysis.

DataFrame `reviews_most_recent` is then stored as a binary file, rather than a `.csv` file. This is mainly because of encoding errors encountered when parsing the review comments after storing then in `.csv` format.

In [ ]:
translated_comments = []

for comment1, comment2 in zip(translated_comments_x, translated_comments_y):
    translated_comments.append(comment1 + " " + comment2)
    
reviews_most_recent["concat_comments"] = translated_comments

jl.dump(reviews_most_recent, 'binary/reviews_most_recent.joblib')

The sentiment analysis for each listing is conducted on a separate `.ipynb` file which is also provided in our work.

The confidence scores for positiveness for the concatenated reviews of each listing are loaded. 

Then, the scores are added as a new column in `reviews_most_recent`.

In [11]:
positiveness = jl.load('binary/sentiment.joblib')
reviews_most_recent["positiveness"] = positiveness

test_listings = test_listings.merge(right=reviews_most_recent, how="left", left_on="id", right_on="listing_id_x")

In [12]:
test_listings = test_listings[["id","prediction","price","prediction_error","positiveness"]]
test_listings["positiveness"] = test_listings["positiveness"].fillna( test_listings["positiveness"].mean() )

Finally, the 1600 top-rated listings are displayed.

In [13]:
test_listings.sort_values("positiveness", ascending=False, ignore_index=True)[["id","positiveness"]].head(1590)

,id,positiveness
0,36306345,1.00
1,39128229,1.00
2,31249607,1.00
3,28277565,1.00
4,38286280,1.00
...,...,...
1585,27304758,1.00
1586,26496151,1.00
1587,25768811,1.00
1588,46848814,1.00


As can be noticed from the above analysis, 1589 listings have the highest positiveness score of 1.0, thus are considered to be the the top-rated listings. 

It should be noted that by keeping more recent reviews for each listing we may be able to acquire a more representative picture for the rating of each listing.